In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import shared_utils

import data_prep
import _utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You mi

In [54]:
import altair as alt
import altair_saver
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [2]:
!pip install cpi

  Using cached cpi-1.0.0-py2.py3-none-any.whl (29.0 MB)


In [3]:
import cpi

In [4]:
pd.set_option("display.max_columns", 100)

## Add adjusted amounts to BC_GTFS_NTD.parquet
adjusting to 2021

In [21]:
df= pd.read_parquet('BC_GTFS_NTD.parquet')

In [22]:
df.sample(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size
493,2021,Section 5311(f),TBD | 0021000227-F,2021,Napa Valley Transportation Authority,BCG0002860,Operating Assistance Sliding Scale,300902,None,275001.0,0.0,0.00,275001.00,0,Open,90088,Full Reporter,66.0,3.0,1.0,8.0,0.0,0.0,0.0,0.0,78.0,7.515625,222854.095200,NaN,0.0,44.0,0.0,2.0,0.0,32.0,0.0,88.0,0.0,32.0,120.0,218.0,0,Ok,Medium
115,2017,Section 5311,CA-2017-168 | 0017000259,2017,City of Rio Vista,BCG0001150,Operating Assistance Sliding Scale,300902,64BO17-00528,68500.0,68500.0,68500.00,0.00,0,Open,9R02-91014,Rural Reporter,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.900000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,6.0,264.0,0,Ok,Small
63,2016,Section 5311,CA-2017-025 | 0017000158,2016,San Diego Metropolitan Transit System,BCG0000086,Rural Operating Assistance,300902,64BO17-00424,265000.0,265000.0,265000.00,0.00,0,Open,90026,Full Reporter,757.0,156.0,40.0,3.0,53.0,1.0,1.0,2.0,1013.0,19.713625,378469.187667,60.0,0.0,598.0,0.0,27.0,164.0,224.0,0.0,1196.0,328.0,224.0,1748.0,278.0,0,Ok,Large
338,2019,Section 5311(f),CA-2018-114 | 0019000012,2019,Lake Transit Authority,BCG0001424,Operating Assistance Sliding Scale - Calistoga...,300902,64CO19-00899,238206.0,238206.0,211540.08,26665.92,0,Open,9R02-91053,Rural Reporter,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,3.260000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,55.0,55.0,159.0,0,Ok,Medium
67,2016,Section 5311,CA-2017-025 | 0017000158,2016,Santa Cruz Metropolitan Transit District,BCG0000250,Operating Assistance,300902,64BO17-00429,168738.0,168738.0,168738.00,0.00,0,Open,90006,Full Reporter,84.0,26.0,14.0,32.0,13.0,1.0,1.0,0.0,171.0,10.224190,296589.723440,47.0,0.0,93.0,0.0,46.0,0.0,32.0,0.0,186.0,0.0,32.0,218.0,296.0,0,Ok,Large


In [23]:
len(df)

798

In [24]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

In [25]:
def adjust_prices(df):
    
    cols =  ["allocationamount",
             "encumbered_amount",
             "expendedamount",
             "activebalance",
             "closedoutbalance"]
    
    ##get cpi table 
    cpi = inflation_table(2021)
    cpi.update
    cpi = (cpi>>select(_.year, _.value))
    cpi_dict = dict(zip(cpi['year'], cpi['value']))
    
    
    for col in cols:
        multiplier = df["project_year"].map(cpi_dict)  
    
        ##using 270.97 for 2021 dollars
        df[f"adjusted_{col}"] = ((df[col] * 270.97) / multiplier)
    return df


In [26]:
adjust_prices(df)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,124939.413803,1.249394e+05,114427.242769,10512.171034,0.0
1,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,20643.0,44265.0,20643.00,0.00,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,23306.112424,4.997554e+04,23306.112424,0.000000,0.0
2,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,23622.0,44265.0,22868.30,753.70,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,26669.427296,4.997554e+04,25818.493956,850.933340,0.0
3,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000286,Purchase Expansion <30ft Bus,111304,64BC17-00480,22925.0,113319.0,22655.51,269.49,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,25882.508710,1.279381e+05,25578.252341,304.256370,0.0
4,2013,Section 5311,CA-18-X059 | 0014000020,2016,Fresno County Rural Transit Agency,BCG0000267,Operating Assistance,300902,64BO17-00399,-769951.0,1552366.0,79018.00,-769951.00,0,Open,9R02-91007,Rural Reporter,117.0,58.0,3.0,0.0,0.0,0.0,0.0,0.0,178.0,5.488889,0.000000,NaN,6.0,21.0,0.0,3.0,0.0,148.0,12.0,42.0,0.0,148.0,202.0,117.0,0,Ok,Large,-869280.849036,1.752634e+06,89211.955214,-869280.849036,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,2021,Section 5311(f),TBD | 0021000227-F,2021,Plumas County Transportation Commission,BCG0002902,Operating Assistance Sliding Scale - 5311f,300902,None,289153.0,0.0,0.00,289153.00,0,Open,9R02-91119,Rural Reporter,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.775000,0.000000,NaN,0.0,4.0,0.0,0.0,0.0,5.0,0.0,8.0,0.0,5.0,13.0,0.0,0,Needed,Small,289153.246255,0.000000e+00,0.000000,289153.246255,0.0
794,2021,Section 5311(f),TBD | 0021000227-F,2021,SunLine Transit Agency,BCG0002948,Operating Assistance Sliding Scale,300902,None,201793.0,0.0,0.00,201793.00,0,Open,90079,Full Reporter,125.0,49.0,4.0,0.0,0.0,0.0,0.0,0.0,178.0,3.469080,114625.362675,NaN,1.0,85.0,0.0,48.0,0.0,44.0,2.0,170.0,0.0,44.0,216.0,327.0,0,Ok,Large,201793.171855,0.000000e+00,0.000000,201793.171855,0.0
795,2021,Section 5311(f),TBD | 0021000227-F,2021,Trinity County,BCG0002937,Operating Assistance Sliding Scale - RED/LEW,300902,None,164715.0,0.0,0.00,164715.00,0,Open,9R02-91035,Rural Reporter,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.583333,0.000000,NaN,0.0,0.0,0.0,1.0,0.0,7.0,0.0,0.0,0.0,7.0,7.0,344.0,0,Ok,Small,164715.140278,0.000000e+00,0.000000,164715.140278,0.0
796,2021,Section 5311(f),TBD | 0021000227-F,2021,Trinity County,BCG0002939,Operating Assistance Sliding Scale - WC,300902,None,143673.0,0

In [67]:
df.grant_fiscal_year.unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [66]:
df.project_year.unique()

array([2016, 2018, 2017, 2019, 2020, 2021])

In [27]:
df>>select(_.project_year, _.allocationamount, _.adjusted_allocationamount)

,project_year,allocationamount,adjusted_allocationamount
0,2016,110663.0,124939.413803
1,2016,20643.0,23306.112424
2,2016,23622.0,26669.427296
3,2016,22925.0,25882.508710
4,2016,-769951.0,-869280.849036
...,...,...,...
793,2021,289153.0,289153.246255
794,2021,201793.0,201793.171855
795,2021,164715.0,164715.140278
796,2021,143673.0,143673.122358


In [28]:
#checking to see if it works. using first row
cpi.inflate(110663.0, 2016)

124939.49388976156

In [124]:
df

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus,name,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,year
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,124939.413803,1.249394e+05,114427.242769,10512.171034,0.0,0.0,Madera County,06 - Fresno,NaN,NaN,Static OK,RT OK,2016-01-01
1,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,20643.0,44265.0,20643.00,0.00,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,23306.112424,4.997554e+04,23306.112424,0.000000,0.0,0.0,Madera County,06 - Fresno,NaN,NaN,Static OK,RT OK,2016-01-01
2,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,23622.0,44265.0,22868.30,753.70,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,26669.427296,4.997554e+04,25818.493956,850.933340,0.0,0.0,Madera County,06 - Fresno,NaN,NaN,Static OK,RT OK,2016-01-01
3,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000286,Purchase Expansion <30ft Bus,111304,64BC17-00480,22925.0,113319.0,22655.51,269.49,0,Open,9R02-91005,Rural Reporter,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,188.0,0,Ok,Medium,25882.508710,1.279381e+05,25578.252341,304.256370,0.0,0.0,Madera County,06 - Fresno,NaN,NaN,Static OK,RT OK,2016-01-01
4,2013,Section 5311,CA-18-X059 | 0014000020,2016,Fresno County Rural Transit Agency,BCG0000267,Operating Assistance,300902,64BO17-00399,-769951.0,1552366.0,79018.00,-769951.00,0,Open,9R02-91007,Rural Reporter,117.0,58.0,3.0,0.0,0.0,0.0,0.0,0.0,178.0,5.488889,0.000000,NaN,6.0,21.0,0.0,3.0,0.0,148.0,12.0,42.0,0.0,148.0,202.0,117.0,0,Ok,Large,-869280.849036,1.752634e+06,89211.955214,-869280.849036,0.0,0.0,Fresno County Rural Transit Agency,06 - Fresno,Fresno Council of Governments,Fresno Council of Governments,Static Incomplete,RT Incomplete,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,2021,Section 5311(f),TBD | 0021000227-F,2021,Plumas County Transportation Commission,BCG0002902,Operating Assistance Sliding Scale - 5311f,300902,None,289153.0,0.0,0.00,289153.00,0,Open,9R02-91119,Rural Reporter,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.775000,0.000000,NaN,0.0,4.0,0.0,0.0,0.0,5.0,0.0,8.0,0.0,5.0,13.0,0.0,0,Needed,Small,289153.246255,0.000000e+00,0.000000,289153.246255,0.0,0.0,Plumas County Transportation Commission,NaN,NaN,NaN,Static OK,RT Incomplete,2021-01-01
794,2021,Section 5311(f),TBD | 0021000227-F,2021,SunLine Transit Agency,BCG0002948,Operating Assistance Sliding Scale,300902,None,201793.0,0.0,0.00,201793.00,0,Open,90079,Full Reporter,125.0,49.0,4.0,0.0,0.0,0.0,0.0,0.0,178.0,3.469080,114

In [126]:
#df.to_parquet("BC_GTFS_NTD_2.parquet")

## Add adjusted amounts to 5311_agg.parquet
 

In [30]:
column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
df['_16plus']= df[column_names].sum(axis=1)

In [31]:
df_agg = df.groupby(['organization_name',]).agg({'allocationamount':'mean',
                                         'encumbered_amount':'mean',
                                         'expendedamount':'mean',
                                         'activebalance':'mean',
                                         'closedoutbalance':'mean',
                                         'expendedamount':'mean',
                                         'adjusted_allocationamount':'mean',
                                         'adjusted_expendedamount':'mean',
                                         'adjusted_encumbered_amount':'mean',
                                         'adjusted_activebalance':'mean',
                                         'adjusted_closedoutbalance':'mean',
                                         'total_vehicles':'max',
                                         'average_age_of_fleet__in_years_':'max',
                                         'average_lifetime_miles_per_vehicle':'max',
                                         'Automobiles':'max',
                                         'Bus':'max',
                                         'Other':'max',
                                         'Train':'max',
                                         'Van':'max',
                                         'automobiles_door':'max',
                                         'bus_doors':'max',
                                         'van_doors':'max',
                                         'train_doors':'max',
                                         'doors_sum':'max',
                                         '_0_9':'max',
                                         '_10_12':'max',
                                         '_13_15':'max',
                                         '_16_20':'max',
                                         '_21_25':'max',
                                         '_26_30':'max',
                                         '_31_60':'max',
                                         '_16plus':'max',
                                         '_60plus':'max'})


df_agg= df_agg.reset_index()

df_agg.head()

,organization_name,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
0,Alpine County Community Development,48766.571429,41909.428571,22843.044286,25923.527143,0.0,51944.783939,24801.626292,45087.635242,27143.157648,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,Amador Regional Transit System,189817.375000,159889.250000,133498.000000,56319.375000,0.0,201514.910754,143955.659727,171586.760266,57559.251027,0.0,24.0,4.065359,0.0000,0.0,0.0,0.0,0.0,21.0,0.0,0.0,21.0,0.0,21.0,18.0,3.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0
2,Butte County Association of Governments,490548.916667,401737.500000,363577.065000,126971.851667,0.0,522308.546176,393332.575684,433497.053873,128975.970491,0.0,53.0,6.228448,200297.8405,0.0,29.0,0.0,0.0,24.0,0.0,58.0,24.0,0.0,82.0,44.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Calaveras Transit Agency,187646.000000,212771.888889,120177.108889,67468.891111,0.0,200172.101735,129096.315133,228709.439943,71075.786602,0.0,12.0,5.000000,0.0000,0.0,0.0,0.0,0.0,11.0,0.0,0.0,11.0,0.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Central Contra Costa Transit Authority, dba: C...",50412.000000,50412.000000,0.000000,50412.000000,0.0,54399.710120,0.000000,54399.710120,54399.710120,0.0,206.0,3.439955,72814.4076,0.0,125.0,0.0,0.0,62.0,0.0,250.0,62.0,0.0,312.0,160.0,43.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0


In [34]:
df4 = (df>>select(_.organization_name, _.reporter_type, _.fleet_size, _.ntd_id, _.itp_id))
df4 = df4.drop_duplicates()
df4

,organization_name,reporter_type,fleet_size,ntd_id,itp_id
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0
4,Fresno County Rural Transit Agency,Rural Reporter,Large,9R02-91007,117.0
8,Yuba-Sutter Transit Authority,Full Reporter,Medium,90061,376.0
10,City of Arcata,Rural Reporter,Medium,9R02-91018,18.0
11,City of Ojai,Rural Reporter,Small,9R02-91058,231.0
...,...,...,...,...,...
542,Fresno Council of Governments,None,No Info,None,0.0
544,Klamath Trinity Non-Emergency Transportation​,None,No Info,None,436.0
547,Yosemite Area Regional Transportation System,Rural Reporter,Medium,9R02-91070,374.0
578,"Greyhound Lines, Inc.",None,No Info,None,0.0


In [35]:
test_merge5 = pd.merge(df4,df_agg, how='left', left_on='organization_name', right_on = 'organization_name')

In [36]:
test_merge5

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0,150308.833333,1.567981e+05,110124.740556,40184.092778,0.0,162562.362848,120648.235973,1.726299e+05,41914.126876,0.0,16.0,5.464286,0.0000,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fresno County Rural Transit Agency,Rural Reporter,Large,9R02-91007,117.0,877618.700000,1.401935e+06,631136.300000,254384.200000,0.0,932754.892597,687760.790181,1.528561e+06,253915.297938,0.0,178.0,5.488889,0.0000,6.0,21.0,0.0,0.0,148.0,12.0,42.0,148.0,0.0,202.0,117.0,58.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Yuba-Sutter Transit Authority,Full Reporter,Medium,90061,376.0,89890.214286,1.358760e+05,52671.892857,37218.321429,0.0,96563.322274,56918.842033,1.495202e+05,39644.480242,0.0,52.0,4.035256,134500.6572,0.0,35.0,0.0,0.0,16.0,0.0,70.0,16.0,0.0,86.0,48.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,City of Arcata,Rural Reporter,Medium,9R02-91018,18.0,82111.428571,1.435400e+05,82111.428571,0.000000,0.0,89821.087756,89821.087756,1.586698e+05,0.000000,0.0,10.0,8.500000,0.0000,0.0,5.0,0.0,0.0,2.0,0.0,10.0,2.0,0.0,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,City of Ojai,Rural Reporter,Small,9R02-91058,231.0,270678.200000,2.263345e+05,216975.417000,53702.783000,0.0,290756.167674,236530.491469,2.464124e+05,54225.676205,0.0,7.0,7.500000,0.0000,0.0,6.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,12.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Fresno Council of Governments,None,No Info,None,0.0,162636.833333,1.436667e+05,127240.670000,35396.163333,0.0,173972.396985,137114.495838,1.550022e+05,36857.901147,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
83,Klamath Trinity Non-Emergency Transportation​,None,No Info,None,436.0,30781.666667,3.078167e+04,2781.333333,28000.333333,0.0,32456.550929,3074.653541,3.245655e+04,29381.897388,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
84,Yosemite Area Regional Transportation System,Rural Reporter,Medium,9R02-91070,374.0,225648.250000,2.006482e+05,188148.250000,37500.000000,0.0,239244.368356,201744.336419,2.155626e+05,37500.031937,0.0,10.0,7.600000,0.0000,0.0,10.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,20.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85,"Greyhound Lines, Inc.",None,No Info,None,0.0,178836.000000,2.560813e+05,217946.769000,-39110.769000,0.0,190996.670780,232838.930648,2.742382e+05,-41842.259867,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [37]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/5311 /"
File_Organization =  "organizations-AllOrganizations_1.csv"
organizations =  pd.read_csv(f'{GCS_FILE_PATH}{File_Organization}')
airtable = to_snakecase(organizations)

airtable = airtable>>select(_.name,
                      # _.ntp_id,
                       # _.itp_id,
                      _.caltrans_district,
                      _.mpo_rtpa,
                      _.planning_authority,
                      _.gtfs_static_status,
                      _.gtfs_realtime_status
                     )    

In [38]:
test_merge6 = pd.merge(test_merge5, airtable, how='left', left_on='organization_name', right_on='name')

In [39]:
test_merge6 = test_merge6.drop('name', 1)

/tmp/ipykernel_847/1989169469.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [40]:
test_merge6

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0,150308.833333,1.567981e+05,110124.740556,40184.092778,0.0,162562.362848,120648.235973,1.726299e+05,41914.126876,0.0,16.0,5.464286,0.0000,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,NaN,NaN,Static OK,RT OK
1,Fresno County Rural Transit Agency,Rural Reporter,Large,9R02-91007,117.0,877618.700000,1.401935e+06,631136.300000,254384.200000,0.0,932754.892597,687760.790181,1.528561e+06,253915.297938,0.0,178.0,5.488889,0.0000,6.0,21.0,0.0,0.0,148.0,12.0,42.0,148.0,0.0,202.0,117.0,58.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,Fresno Council of Governments,Fresno Council of Governments,Static Incomplete,RT Incomplete
2,Yuba-Sutter Transit Authority,Full Reporter,Medium,90061,376.0,89890.214286,1.358760e+05,52671.892857,37218.321429,0.0,96563.322274,56918.842033,1.495202e+05,39644.480242,0.0,52.0,4.035256,134500.6572,0.0,35.0,0.0,0.0,16.0,0.0,70.0,16.0,0.0,86.0,48.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,03 - Marysville,Sacramento Area Council of Governments,Sacramento Area Council of Governments,Static OK,RT Incomplete
3,City of Arcata,Rural Reporter,Medium,9R02-91018,18.0,82111.428571,1.435400e+05,82111.428571,0.000000,0.0,89821.087756,89821.087756,1.586698e+05,0.000000,0.0,10.0,8.500000,0.0000,0.0,5.0,0.0,0.0,2.0,0.0,10.0,2.0,0.0,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,01 - Eureka,NaN,Humboldt County Association of Governments,Static OK,RT OK
4,City of Ojai,Rural Reporter,Small,9R02-91058,231.0,270678.200000,2.263345e+05,216975.417000,53702.783000,0.0,290756.167674,236530.491469,2.464124e+05,54225.676205,0.0,7.0,7.500000,0.0000,0.0,6.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,12.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07 - Los Angeles,Southern California Association of Governments,Southern California Association of Governments,Static OK,RT OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Fresno Council of Governments,None,No Info,None,0.0,162636.833333,1.436667e+05,127240.670000,35396.163333,0.0,173972.396985,137114.495838,1.550022e+05,36857.901147,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Static OK,RT Incomplete
83,Klamath Trinity Non-Emergency Transportation​,None,No Info,None,436.0,30781.666667,3.078167e+04,2781.333333,28000.333333,0.0,32456.550929,3074.653541,3.245655e+04,29381.897388,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
84,Yosemite Area Regional Transportation System,Rural Reporter,Medium,9R02-91070,374.0,225648.250000,2.006482e+05,188148.250000,37500.000000,0.0,239244.368356,201744.336419,2.155626e+05,37500.031937,0.0,10.0,7.600000,0.0000,0.0,10.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,20.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10 - Stockton,Merced County Association of Governments,Merced County Association of Governments,Static OK,RT Incomplete
85,"Greyhound Lines, Inc.",None,No Info,None,0.0,178836.000000,2.560813e+05,217946.769000,-39110.769000,0.0,190996.670780,232838.930648,2.742382e+05,-41842.259867,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
#test_merge6.to_parquet("5311_agg.parquet")